In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')
path = "/BraTS20-Invariant-feature"
os.chdir(path)
os.getcwd()

In [ ]:
pip install pyradiomics

In [ ]:
import pathlib
import imageio.v2 as imageio
import SimpleITK as sitk
import matplotlib.pyplot as plt
from radiomics import featureextractor
import pandas as pd
import re
import numpy as np
from radiomics import glszm

In [ ]:
directory = pathlib.Path("/BraTS20-Invariant-feature/BraTS2020_Post")

def subfolders(path):
    if not os.path.exists(path):
        return 0, []  # Path doesn't exist, so no subfolders.

    if not os.path.isdir(path):
        return 0, []  # Path is not a directory, so no subfolders.

    subfolders = [f for f in os.listdir(path) if os.path.isdir(os.path.join(path, f))]
    return len(subfolders), subfolders

subfolder_count, sub_folders = subfolders(directory)
print(f"Number of subfolders: {subfolder_count}")

In [ ]:
#extract all original texture features from pyradiomics
dir = "/BraTS20-Invariant-feature/BraTS2020_Post/"

extractor = featureextractor.RadiomicsFeatureExtractor()
radiomic_features = {}

for folder_name in sub_folders:
  image_path = dir + folder_name + "/" + folder_name + '_t2.jpg'
  mask_path = dir + folder_name + "/" + folder_name + '_mask.jpg'
  image = imageio.imread(str(image_path))
  mask = imageio.imread(str(mask_path))
  image = sitk.GetImageFromArray(image[:, :, 2])
  mask = sitk.GetImageFromArray((mask[:, :, 1] > 240).astype(np.uint8))
  result = extractor.execute(image, mask)
  radiomic_features[str(folder_name)] = result

radiomic_features = pd.DataFrame.from_dict(radiomic_features, orient='index')
radiomic_features = radiomic_features[radiomic_features.columns.drop(list(radiomic_features.filter(regex='diagnostics')))]
radiomic_features = radiomic_features[radiomic_features.columns.drop(list(radiomic_features.filter(regex='firstorder')))]

# Sort the DataFrame based on the index
radiomic_features = radiomic_features.sort_index()

# Save the sorted DataFrame to Excel
radiomic_features.to_excel('original_radiomic_features_BraTS2020.xlsx')